# House Price Preprocessing of data: Ames dataset

The Ames Housing dataset contains detailed information on residential homes sold in Ames,  between 2006 and 2010. It is known for its rich feature set and real-world complexity, making it an excellent choice to demonstrate end to end modelling process.

- 81 features per home sale record
- A mix of categorical, ordinal, and numerical variables
- Includes house characteristics, quality ratings, and sale prices
- Contains missing values, outliers, and inconsistent entries

We will:

* Load the Ames Housing dataset
* Explore the dataset to understand the structure of the data
* Preprocess the data by handling the typos, missing values, removing columns with high missing rates, handling outliers, transforming variables, encoding categorical variables and many more
* Predict the salesprice in the Ames dataset.


# 1. Loading the dataset

In [21]:
#Had to install these packages because they were not part of my current enviroment

#pip install pyDOE
#!pip install rapidfuzz

In [22]:
# Importing essential libraries
from pyDOE import fullfact
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
import difflib
from rapidfuzz import fuzz, process


In [23]:
# Setting display options for better readability
pd.set_option('display.max_columns', None)

# Loading the dataset
df = pd.read_csv('Ames_Housing_Data.csv',sep=';')

# Check shape of the dataset
print(f"The dataset contains {df.shape[0]} rows and {df.shape[1]} columns.")



The dataset contains 2930 rows and 81 columns.


# 2. Explore the data

In [24]:
# Displaying the first five rows
df.head()


,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [25]:

# View column types and non-null counts
df.info()
df.shape


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PID              2930 non-null   int64  
 1   MS SubClass      2930 non-null   int64  
 2   MS Zoning        2930 non-null   object 
 3   Lot Frontage     2440 non-null   float64
 4   Lot Area         2930 non-null   int64  
 5   Street           2930 non-null   object 
 6   Alley            198 non-null    object 
 7   Lot Shape        2930 non-null   object 
 8   Land Contour     2930 non-null   object 
 9   Utilities        2930 non-null   object 
 10  Lot Config       2930 non-null   object 
 11  Land Slope       2930 non-null   object 
 12  Neighborhood     2930 non-null   object 
 13  Condition 1      2930 non-null   object 
 14  Condition 2      2930 non-null   object 
 15  Bldg Type        2930 non-null   object 
 16  House Style      2930 non-null   object 
 17  Overall Qual  

(2930, 81)

# 3. Preprocessing
 
- 3.1 Handling inconsistencies within the data
- 3.2 Removing duplicates and fixing inconsistent types
- 3.3 Identifying and handling missing values
- 3.4 Outlier treatment and visual validation
- 3.5 Feature engineering
     * 3.5.1 Log transform the numerical features
     * 3.5.2 Encoding the categorical variables using
     * 3.5.3 Creating additional features 
     * 3.5.4 Scalling the numerical features using StandardScaler
- 3.6 Feature Selection


3.1 Handling inconsistencies within the data

    - Clean categorical columns in Ames Housing dataset by:
    - Converting to lowercase
    - Stripping spaces
    - Automatically detecting & fixing typos

    

In [26]:
def review_typos_fast(df, similarity_threshold=85):

    dfs = df.copy()

    #Dictionary to Store Results
    suggestions = {}

    #Run typo detection on categorical variables.
    cat_cols = dfs.select_dtypes(include=['object', 'category']).columns
    
    #Processes one column at a time
    for col in cat_cols:
        
        # Standardize formatting: Lowercase and removes extra spaces like
        dfs[col] = dfs[col].astype(str).str.lower().str.strip()
        
        #Gets unique values in the column (ignores NaN) and sorts them for consistency.
        unique_vals = sorted(dfs[col].dropna().unique())

        #List to store similar-value groups 
        groups = []

        #Keeps track of values we’ve already checked so we don’t repeat comparisons.
        visited = set()
        
        #Checks each value only if we haven’t already grouped it.
        for val in unique_vals:
            if val not in visited:

                #Compares val to all other values in unique_vals and Measures similarity 
                matches = process.extract(val, unique_vals, scorer=fuzz.ratio, limit=None)

                #Keeps only matches above similarity_threshold
                close_matches = [m[0] for m in matches if m[1] >= similarity_threshold]
                
                #If there’s more than 1 similar value, we have a possible typo group and adds the group to groups.
                if len(close_matches) > 1:
                    groups.append(close_matches)

                #Marks all values in the group as processed so we don’t re-check them.
                visited.update(close_matches)
        
        #If any typo groups were found, save them to suggestions under the column name
        if groups:
            suggestions[col] = groups
    
    # Display suggestions
    for col, groups in suggestions.items():
        print(f"\n Column: {col}")
        for group in groups:
            print(f"Possible typo group: {group}")
    
    return suggestions

# 🔹 Usage
typo_suggestions = review_typos_fast(df)




 Column: MS Zoning
Possible typo group: ['c (all)', 'i (all)']

 Column: Neighborhood
Possible typo group: ['names', 'nwames']
Possible typo group: ['sawyer', 'sawyerw']

 Column: Bldg Type
Possible typo group: ['twnhs', 'twnhse']

 Column: Exterior 2nd
Possible typo group: ['wd sdng', 'wd shng']


In [27]:

# Replace typos automatically
typo_suggestions = {
    'Neighborhood': [['nwames', 'names'], ['sawyerw', 'sawyer']],
    'Bldg Type': [['twnhse', 'twnhs']],
    'Exterior 2nd': [['wd shng', 'wd sdng']]
}

def apply_typos(dfs, typo_suggestions):
        
    df_corrected = dfs.copy()
    corrections_log = []
    
    for col, groups in typo_suggestions.items():
        for group in groups:
            correct_value = max(group, key=lambda x: df_corrected[col].str.lower().str.strip().tolist().count(x))
            df_corrected[col] = df_corrected[col].replace(group, correct_value)
            
            corrections_log.append({
                "Column": col,
                "Replaced Values": group,
                "Correct Value": correct_value
            })
    
    print("Typo corrections applied!")
    return df_corrected, corrections_log

# Usage
df_corrected, corrections_log = apply_typos(df, typo_suggestions)

# View what was changed
for log in corrections_log:
    print(f"\nColumn: {log['Column']}")
    print(f"Replaced: {log['Replaced Values']} → {log['Correct Value']}")


Typo corrections applied!

Column: Neighborhood
Replaced: ['nwames', 'names'] → names

Column: Neighborhood
Replaced: ['sawyerw', 'sawyer'] → sawyer

Column: Bldg Type
Replaced: ['twnhse', 'twnhs'] → twnhse

Column: Exterior 2nd
Replaced: ['wd shng', 'wd sdng'] → wd sdng



3.2 Removing duplicates and fixing inconsistent types

    - Duplicates can skew analysis and bias model results
   

In [28]:
# Check for duplicate rows
num_dup = df_corrected.duplicated().sum()

# Drop duplicates.Use subset attribute to drop duplicate based on specific columns
df_corrected = df_corrected.drop_duplicates()
print("Duplicates Dropped:", num_dup)


Duplicates Dropped: 0



3.3 Identifying and handling missing values (Imputation)

    - Drop colums with 80% or more missing
    - Determine the imputation method (mean/median) for numerical features based on skewness of data and mode method for categorical features


In [29]:
# Total missing values per column: TOP 28
df_corrected.isnull().sum().sort_values(ascending=False).head(28) 

Pool QC           2917
Misc Feature      2824
Alley             2732
Fence             2358
Mas Vnr Type      1775
Fireplace Qu      1422
Lot Frontage       490
Garage Cond        159
Garage Finish      159
Garage Yr Blt      159
Garage Qual        159
Garage Type        157
Bsmt Exposure       83
BsmtFin Type 2      81
Bsmt Qual           80
Bsmt Cond           80
BsmtFin Type 1      80
Mas Vnr Area        23
Bsmt Half Bath       2
Bsmt Full Bath       2
BsmtFin SF 1         1
Garage Cars          1
Electrical           1
Total Bsmt SF        1
Bsmt Unf SF          1
BsmtFin SF 2         1
Garage Area          1
Sale Condition       0
dtype: int64

* First determine which coloumns to drop and drop them

In [30]:
# Make a Function to calculate the percent of missing data in each columns (feature) and then sort it
def missing_percent(df_corrected):
    nan_percent= 100*(df_corrected.isnull().sum()/len(df_corrected))
    nan_percent= nan_percent[nan_percent>0].sort_values()
    return nan_percent
nan_percent= missing_percent(df_corrected)
nan_percent

# Calculate the percentage of missing values for each column
missing_percent = df_corrected.isnull().mean() * 100

# Print out the missing percentage for each column
print("Missing percentage for each column:")
print(missing_percent)


Missing percentage for each column:
PID                0.000000
MS SubClass        0.000000
MS Zoning          0.000000
Lot Frontage      16.723549
Lot Area           0.000000
                    ...    
Mo Sold            0.000000
Yr Sold            0.000000
Sale Type          0.000000
Sale Condition     0.000000
SalePrice          0.000000
Length: 81, dtype: float64


In [31]:

# Identify columns with more than 80% missing values
cols_above_80 = missing_percent[missing_percent > 80].index.tolist()
print("Columns with more than 80% missing values:", cols_above_80)


Columns with more than 80% missing values: ['Alley', 'Pool QC', 'Fence', 'Misc Feature']


In [32]:

# List of columns with more than 80% missing values and drop them
cols_to_drop = ['Alley', 'Pool QC', 'Fence', 'Misc Feature']
df_corrected.drop(columns=cols_to_drop, inplace=True, errors='ignore')
print("Dropped columns with more than 80% missing values.")


Dropped columns with more than 80% missing values.


Imputation:

Use skewness to determine whether to use Mean or Median to impute the numerical features
* |Skewness| < 0.5 → Strongly symmetric → Use Mean: The mean is affected by extreme values.
* 0.5 ≤ |Skewness| ≤ 1 → Mild skew → Either Mean or Median could work
* |Skewness| > 1 → Strong skew → Use Median: The median is robust — outliers don’t pull it much.

In [33]:
#%whos

In [34]:

# Create summary list
summary = []

for col in df_corrected:
    col_data = df_corrected[col]
    
    # Numeric columns
    if pd.api.types.is_numeric_dtype(col_data):
        mean_val = col_data.mean()
        median_val = col_data.median()
        mode_val = col_data.mode().iloc[0] if not col_data.mode().empty else None
        skewness = col_data.skew()
        
        summary.append({"Column": col,"Type": "Numeric","Missing %": col_data.isnull().mean() * 100,"Mean": mean_val,"Median": median_val,"Mode": mode_val,"Skewness": skewness})
    
    # Categorical columns
    else:
        mode_val = col_data.mode().iloc[0] if not col_data.mode().empty else None
        
        summary.append({"Column": col,"Type": "Categorical","Missing %": col_data.isnull().mean() * 100,"Mean": None,"Median": None,"Mode": mode_val,"Skewness": None })
        
# Convert summary to DataFrame
summary_df = pd.DataFrame(summary).sort_values(by="Skewness", ascending=False)

# Display summary table
print(summary_df)


             Column         Type  Missing %          Mean  Median    Mode  \
71         Misc Val      Numeric   0.000000     50.635154     0.0       0   
70        Pool Area      Numeric   0.000000      2.243345     0.0       0   
4          Lot Area      Numeric   0.000000  10147.921843  9436.5    9600   
44  Low Qual Fin SF      Numeric   0.000000      4.676792     0.0       0   
68       3Ssn Porch      Numeric   0.000000      2.592491     0.0       0   
..              ...          ...        ...           ...     ...     ...   
62      Garage Qual  Categorical   5.426621           NaN     NaN      TA   
63      Garage Cond  Categorical   5.426621           NaN     NaN      TA   
64      Paved Drive  Categorical   0.000000           NaN     NaN       Y   
74        Sale Type  Categorical   0.000000           NaN     NaN     WD    
75   Sale Condition  Categorical   0.000000           NaN     NaN  Normal   

     Skewness  
71  21.999788  
70  16.939142  
4   12.820898  
44  12.1181

In [35]:
# Create a new column in summary_df to record method & value used
summary_df["Imputation Method"] = None
summary_df["Imputation Value"] = None

for i, row in summary_df.iterrows():
    col = row['Column']
    col_type = row['Type']
    skewness = row['Skewness']
    
    # Numeric columns → Mean or Median based on skewness
    if col_type == "Numeric":
        if abs(skewness) > 1:  # Strong skew → Median
            value = df_corrected[col].median()
            df_corrected[col]=df_corrected[col].fillna(value)
            method = "Median"
        else:  # Low skew → Mean
            value = df_corrected[col].mean()
            df_corrected[col]=df_corrected[col].fillna(value)
            method = "Mean"
    
    # Categorical columns → Mode
    else:
        value = df_corrected[col].mode().iloc[0] if not df_corrected[col].mode().empty else None
        df_corrected[col]=df_corrected[col].fillna(value)
        method = "Mode"
    
    # Update summary_df with method & value used
    summary_df.at[i, "Imputation Method"] = method
    summary_df.at[i, "Imputation Value"] = value
    
# View updated summary_df
from IPython.display import display
display(summary_df)


,Column,Type,Missing %,Mean,Median,Mode,Skewness,Imputation Method,Imputation Value
71,Misc Val,Numeric,0.000000,50.635154,0.0,0,21.999788,Median,0.0
70,Pool Area,Numeric,0.000000,2.243345,0.0,0,16.939142,Median,0.0
4,Lot Area,Numeric,0.000000,10147.921843,9436.5,9600,12.820898,Median,9436.5
44,Low Qual Fin SF,Numeric,0.000000,4.676792,0.0,0,12.118162,Median,0.0
68,3Ssn Porch,Numeric,0.000000,2.592491,0.0,0,11.403795,Median,0.0
...,...,...,...,...,...,...,...,...,...
62,Garage Qual,Categorical,5.426621,NaN,NaN,TA,NaN,Mode,TA
63,Garage Cond,Categorical,5.426621,NaN,NaN,TA,NaN,Mode,TA
64,Paved Drive,Categorical,0.000000,NaN,NaN,Y,NaN,Mode,Y
74,Sale Type,Categorical,0.000000,NaN,NaN,WD,NaN,Mode,WD


* Checking results

In [36]:
print("\nMissing values after imputation:")
print(df_corrected.isnull().sum())


Missing values after imputation:
PID               0
MS SubClass       0
MS Zoning         0
Lot Frontage      0
Lot Area          0
                 ..
Mo Sold           0
Yr Sold           0
Sale Type         0
Sale Condition    0
SalePrice         0
Length: 77, dtype: int64



3.4 Outlier treatment and visual validation

    - Detect outliers in all numeric columns using IQR and percentile method and and apply Winsorization to cap based on skewness

In [37]:

# Select numeric columns
numeric_cols = df_corrected.select_dtypes(include=[np.number]).columns.drop('SalePrice', errors='ignore')

# Winsorization functions
def iqr_winsor(series, k=1.5):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - k * IQR
    upper = Q3 + k * IQR
    winsorized = np.clip(series, lower, upper)
    capped_percent = (series != winsorized).sum() / len(series) * 100
    return winsorized, capped_percent

def percentile_winsor(series, lower_pct=0.01, upper_pct=0.99):
    lower = series.quantile(lower_pct)
    upper = series.quantile(upper_pct)
    winsorized = np.clip(series, lower, upper)
    capped_percent = (series != winsorized).sum() / len(series) * 100
    return winsorized, capped_percent

# Skewness threshold
skew_threshold = 1  #or 0.75 if you prefer

# Dataframe for winsorized data
df_winsor = df_corrected.copy()
summary_log = []

# Loop over numeric columns
for col in numeric_cols:
    skewness = df_corrected[col].skew()
    
    if abs(skewness) > skew_threshold:
        winsorized, capped_percent = percentile_winsor(df_corrected[col])
        if capped_percent > 0:
            df_winsor[col] = winsorized
            method = "Percentile"
        else:
            method = "No outliers detected"
    else:
        winsorized, capped_percent = iqr_winsor(df_corrected[col])
        if capped_percent > 0:
            df_winsor[col] = winsorized
            method = "IQR"
        else:
            method = "No outliers detected"
    
    summary_log.append({
        "Variable": col,
        "Skewness": round(skewness, 2),
        "Method": method,
        "Percent_Capped": round(capped_percent, 2)
    })

# Create summary table
summary_dfcorrected = pd.DataFrame(summary_log).sort_values(by="Percent_Capped", ascending=False)

# Show summary
print("\nSummary of Winsorization Applied:")
print(summary_dfcorrected)



Summary of Winsorization Applied:
           Variable  Skewness                Method  Percent_Capped
5      Overall Cond      0.57                   IQR            8.60
21    Bedroom AbvGr      0.31                   IQR            2.66
16      Gr Liv Area      1.27            Percentile            2.05
13       1st Flr SF      1.47            Percentile            2.01
11      Bsmt Unf SF      0.92                   IQR            1.91
23    TotRms AbvGrd      0.75                   IQR            1.74
3          Lot Area     12.82            Percentile            1.67
27      Garage Area      0.24                   IQR            1.43
29    Open Porch SF      2.54            Percentile            1.02
2      Lot Frontage      1.67            Percentile            1.02
28     Wood Deck SF      1.84            Percentile            1.02
8      Mas Vnr Area      2.62            Percentile            1.02
34         Misc Val     22.00            Percentile            1.02
10     BsmtFi

c:\Users\Pena\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return bound(*args, **kwds)


In [38]:

df_winsor.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,526301100,20,RL,132.42,31770.0,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5.0,1960.0,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656.0,0.0,0.0,1656.0,1.0,0.0,1.0,0,3.0,1,TA,7,Typ,2.0,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210.0,62.0,0,0,0.0,0,0,5,2010,WD,Normal,215000
1,526350040,20,RH,80.00,11622.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6.0,1961.0,1961,Gable,CompShg,VinylSd,VinylSd,BrkFace,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896.0,0.0,0.0,896.0,0.0,0.0,1.0,0,2.0,1,TA,5,Typ,0.0,Gd,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140.0,0.0,0,0,120.0,0,0,6,2010,WD,Normal,105000
2,526351010,20,RL,81.00,14267.0,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6.0,1958.0,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329.0,0.0,0.0,1329.0,0.0,0.0,1.0,1,3.0,1,Gd,6,Typ,0.0,Gd,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393.0,36.0,0,0,0.0,0,971,6,2010,WD,Normal,172000
3,526353030,20,RL,93.00,11160.0,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5.0,1968.0,1968,Hip,CompShg,BrkFace,BrkFace,BrkFace,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110.0,0.0,0.0,2110.0,1.0,0.0,2.0,1,3.0,1,Ex,8,Typ,2.0,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0.0,0.0,0,0,0.0,0,0,4,2010,WD,Normal,244000
4,527105010,60,RL,74.00,13830.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5.0,1997.0,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928.0,701.0,0.0,1629.0,0.0,0.0,2.0,1,3.0,1,TA,6,Typ,1.0,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212.0,34.0,0,0,0.0,0,0,3,2010,WD,Normal,189900


* Visual Validation

In [39]:
#When you run the visual validation code, out comment %%skip below
%%skip

# Visualize variables where winsorization was applied
vars_with_outliers = summary_dfcorrected[summary_dfcorrected["Method"] != "No outliers detected"]["Variable"]

for col in vars_with_outliers:
    plt.figure(figsize=(10,4))
    
    plt.subplot(1,2,1)
    sns.boxplot(x=df[col], color="lightcoral")
    plt.title(f"{col} - Before Winsorization")
    
    plt.subplot(1,2,2)
    sns.boxplot(x=df_winsor[col], color="lightgreen")
    plt.title(f"{col} - After Winsorization")
    
    plt.tight_layout()
    plt.show()

UsageError: Line magic function `%%skip` not found.


3.5 Feature Engineering

3.5.1 Log transform the numerical features using log(x+1)

    - Log transformation compresses large values and makes the distribution more symmetric, which helps:
      * Models that assume normality (linear regression, GLM)
      * Stabilize variance
      * Reduce leverage of large values

Threshold reasoning:
* |skewness| < 1 → Distribution is fairly symmetrical → No transformation needed
* |skewness| ≥ 1 → High skewness → Transformation strongly recommended (log, sqrt, Box-Cox)


In [40]:
# Copy winsorized dataframe and drop the target feauture
df_log = df_winsor.copy().drop('SalePrice', errors='ignore')

# Create a log transformation log
log_transform_log = []

for col in numeric_cols:
    skew_after = df_log[col].skew()
    
    if abs(skew_after) > skew_threshold:
        # Apply log1p transformation safely
        df_log[col] = np.log1p(df_log[col])
        log_transform_log.append({
            "Variable": col,
            "Skewness_After_Winsor": round(skew_after, 2),
            "Log_Transform_Applied": "Yes"
        })
    else:
        log_transform_log.append({
            "Variable": col,
            "Skewness_After_Winsor": round(skew_after, 2),
            "Log_Transform_Applied": "No"
        })

# Create log summary dataframe
log_summary_df = pd.DataFrame(log_transform_log)

# Example: ensure 'Variable' is string type for both dataframes
summary_dfcorrected['Variable'] = summary_dfcorrected['Variable'].astype(str)
log_summary_df['Variable'] = log_summary_df['Variable'].astype(str)

# Merge on Variable
final_summary_df = summary_dfcorrected.merge(log_summary_df[['Variable', 'Log_Transform_Applied']], on='Variable', how='left')

# Display final summary
print("\nFinal Summary (Winsorization + Log Transform):")
pd.set_option('display.max_rows', None)  # show all rows
display(final_summary_df.sort_values(by="Log_Transform_Applied", ascending=False))
pd.reset_option('display.max_rows')  # reset option back


Final Summary (Winsorization + Log Transform):


,Variable,Skewness,Method,Percent_Capped,Log_Transform_Applied
18,3Ssn Porch,11.40,Percentile,0.99,Yes
10,Wood Deck SF,1.84,Percentile,1.02,Yes
26,Kitchen AbvGr,4.31,Percentile,0.17,Yes
19,Enclosed Porch,4.01,Percentile,0.96,Yes
16,Screen Porch,3.96,Percentile,1.02,Yes
15,Low Qual Fin SF,12.12,Percentile,1.02,Yes
29,Bsmt Half Bath,3.94,Percentile,0.14,Yes
13,BsmtFin SF 2,4.14,Percentile,1.02,Yes
12,Misc Val,22.00,Percentile,1.02,Yes
11,Mas Vnr Area,2.62,Percentile,1.02,Yes


3.5.2 Encoding the categorical variables

    - One-Hot Encoding is great because: 
      * Avoids false ordinal relationships
      * Makes categorical data usable in most models
      * Keeps interpretability clean

In [43]:
df0 = df_log.copy()

# Optional: separate target if present
target_col = 'SalePrice'
y = None
if target_col in df0.columns:
    y = df0[target_col].copy()

# Encode categoricals (one-hot)
categorical_cols = df0.select_dtypes(include=['object', 'category']).columns.tolist()

# Keep a snapshot of columns pre-encoding to identify new dummy columns
cols_before_encoding = df0.columns.tolist()

df_encoded = pd.get_dummies(
    df0,
    columns=categorical_cols,
    drop_first=True,
    dtype=np.uint8  # keeps compact 0/1 ints
)

# One-hot columns are the new columns added by get_dummies
onehot_cols = [c for c in df_encoded.columns if c not in cols_before_encoding]

# Summary for your records (optional)
encoding_summary = pd.DataFrame({
    'Variable': categorical_cols,
    'Encoding': 'One-Hot Encoding',
    'Reason': 'Nominal categorical variable (default approach)'
})


In [44]:
display(encoding_summary)

,Variable,Encoding,Reason
0,MS Zoning,One-Hot Encoding,Nominal categorical variable (default approach)
1,Street,One-Hot Encoding,Nominal categorical variable (default approach)
2,Lot Shape,One-Hot Encoding,Nominal categorical variable (default approach)
3,Land Contour,One-Hot Encoding,Nominal categorical variable (default approach)
4,Utilities,One-Hot Encoding,Nominal categorical variable (default approach)
5,Lot Config,One-Hot Encoding,Nominal categorical variable (default approach)
6,Land Slope,One-Hot Encoding,Nominal categorical variable (default approach)
7,Neighborhood,One-Hot Encoding,Nominal categorical variable (default approach)
8,Condition 1,One-Hot Encoding,Nominal categorical variable (default approach)
9,Condition 2,One-Hot Encoding,Nominal categorical variable (default approach)


3.5.3 Creating additional features

    - Age feature
    - Polynomial transformation

In [46]:
# Age features (e.g., Yr Sold - Year Built*)

df_fe = df_encoded.copy()
feature_summary = []

# Numeric columns at this stage (includes one-hot, we’ll exclude those explicitly)
numeric_all_now = df_fe.select_dtypes(include=[np.number]).columns.tolist()

# True numeric (exclude target & one-hot)
true_numeric_base = [
    c for c in numeric_all_now
    if c != target_col and c not in onehot_cols
]

# Age features: try common patterns, only create if both columns exist and numeric
age_cols_created = []
yr_sold_candidates = [c for c in df_fe.columns if c.lower().replace('_',' ').strip() in {'yr sold', 'year sold', 'sale year'}]
if not yr_sold_candidates and 'Yr Sold' in df_fe.columns:
    yr_sold_candidates = ['Yr Sold']

if yr_sold_candidates:
    yr_sold_col = yr_sold_candidates[0]
    # Examples of year-built-like columns
    year_built_like = [c for c in df_fe.columns if any(k in c.lower() for k in ['year built','yr built','year_built','yr_built','year remod','remod','renov'])]
    # If you had a specific name e.g. "Year Built", add it to be safe
    if 'Year Built' in df_fe.columns and 'Year Built' not in year_built_like:
        year_built_like.append('Year Built')

    for col in year_built_like:
        if col in df_fe.columns and np.issubdtype(df_fe[col].dtype, np.number) and np.issubdtype(df_fe[yr_sold_col].dtype, np.number):
            new_col = f'{col}_Age'
            df_fe[new_col] = df_fe[yr_sold_col] - df_fe[col]
            age_cols_created.append(new_col)
            feature_summary.append({
                'Feature': new_col,
                'Formula': f'{yr_sold_col} - {col}',
                'Reason': 'Captures property age / time since built/remodel'
            })

# Polynomial interactions (degree=2) on true numeric only

poly_features = PolynomialFeatures(degree=2, include_bias=False)
# Fit-transform on the base true numeric (before adding interactions), guard empty case
if true_numeric_base:
    poly_array = poly_features.fit_transform(df_fe[true_numeric_base])
    poly_df = pd.DataFrame(
        poly_array,
        columns=poly_features.get_feature_names_out(true_numeric_base),
        index=df_fe.index
    )

    # Keep only interaction terms (feature names with a space like "A B")
    interaction_cols = [c for c in poly_df.columns if ' ' in c]
    if interaction_cols:
        df_fe = pd.concat([df_fe, poly_df[interaction_cols]], axis=1)
        for col in interaction_cols:
            feature_summary.append({
                'Feature': col,
                'Formula': 'Pairwise interaction of numeric variables',
                'Reason': 'Capture 2-way non-linear relationships'
            })
else:
    interaction_cols = []

feature_summary_df = pd.DataFrame(feature_summary) if feature_summary else pd.DataFrame(columns=['Feature','Formula','Reason'])


3.5.4 Scalling the numerical features using StandardScaler

    - Remove magnitude bias
    - Speed up learning of mofrl
    - Stabilize optimization
    - Make features comparable
   

In [47]:

# Scale ONLY true numeric features
numeric_all_after = df_fe.select_dtypes(include=[np.number]).columns.tolist()

# Identify all one-hot columns again (they’re still the same set we captured), and target
# True numeric for scaling: exclude one-hots & target
true_numeric_for_scaling = [
    c for c in numeric_all_after
    if c not in onehot_cols and c != target_col
]

# If you prefer not to scale the newly created interaction terms, you can exclude them here:
# true_numeric_for_scaling = [c for c in true_numeric_for_scaling if c not in interaction_cols]

scaler = StandardScaler()
if true_numeric_for_scaling:
    df_fe[true_numeric_for_scaling] = scaler.fit_transform(df_fe[true_numeric_for_scaling])

    # Build aligned scaling params from what the scaler actually saw (guarantees lengths match)
    try:
        feature_names_scaled = list(scaler.feature_names_in_)  # sklearn >= 1.0
    except AttributeError:
        feature_names_scaled = true_numeric_for_scaling  # fallback, same order as fit

    scaling_params = pd.DataFrame({
        'Feature': feature_names_scaled,
        'Mean': scaler.mean_,
        'StdDev': scaler.scale_
    })
else:
    scaling_params = pd.DataFrame(columns=['Feature','Mean','StdDev'])

# -------------------------
# 5) Final outputs
# -------------------------
df_prepared = df_fe.copy()
# Reattach target if you had separated it (optional)
if y is not None:
    df_prepared[target_col] = y

print("Encoding done (one-hot).")
print(f"One-hot columns: {len(onehot_cols)}")
print(f"Age features created: {age_cols_created}")
print(f"Interaction terms added: {len(interaction_cols)}")
print(f"Scaled true numeric features: {len(true_numeric_for_scaling)}")
print("\nScaling parameters (head):")
print(scaling_params.head())


Encoding done (one-hot).
One-hot columns: 213
Age features created: ['Year Built_Age', 'Year Remod/Add_Age']
Interaction terms added: 736
Scaled true numeric features: 775

Scaling parameters (head):
        Feature          Mean        StdDev
0           PID  7.144645e+08  1.886986e+08
1   MS SubClass  3.824262e+00  6.937428e-01
2   MS SubClass  3.824262e+00  6.937428e-01
3  Lot Frontage  6.870225e+01  1.961954e+01
4  Lot Frontage  6.870225e+01  1.961954e+01


3.5.2 Encoding the categorical variables using:

    - One-Hot Encoding for nominal categorical variables with relatively few categories.


In [48]:
# Copy dataframe after log transform
df_onehot = df_log.copy()

# Detect all categorical columns
categorical_cols = df_onehot.select_dtypes(include=['object', 'category']).columns

# Apply one-hot encoding (force 0/1 integers)
df_onehot_encoded = pd.get_dummies(df_onehot, columns=categorical_cols, drop_first=True, dtype=int)

# Summary table
encoding_summary = pd.DataFrame({
    'Variable': categorical_cols,
    'Encoding': 'One-Hot Encoding',
    'Reason': 'Nominal categorical variable (default approach)'
})

# Display summary
print("\nEncoding Summary:")
print(encoding_summary)


Encoding Summary:
          Variable          Encoding  \
0        MS Zoning  One-Hot Encoding   
1           Street  One-Hot Encoding   
2        Lot Shape  One-Hot Encoding   
3     Land Contour  One-Hot Encoding   
4        Utilities  One-Hot Encoding   
5       Lot Config  One-Hot Encoding   
6       Land Slope  One-Hot Encoding   
7     Neighborhood  One-Hot Encoding   
8      Condition 1  One-Hot Encoding   
9      Condition 2  One-Hot Encoding   
10       Bldg Type  One-Hot Encoding   
11     House Style  One-Hot Encoding   
12      Roof Style  One-Hot Encoding   
13       Roof Matl  One-Hot Encoding   
14    Exterior 1st  One-Hot Encoding   
15    Exterior 2nd  One-Hot Encoding   
16    Mas Vnr Type  One-Hot Encoding   
17      Exter Qual  One-Hot Encoding   
18      Exter Cond  One-Hot Encoding   
19      Foundation  One-Hot Encoding   
20       Bsmt Qual  One-Hot Encoding   
21       Bsmt Cond  One-Hot Encoding   
22   Bsmt Exposure  One-Hot Encoding   
23  BsmtFin Type 1  O

In [49]:

df_onehot_encoded.head()

,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Street_Pave,Lot Shape_IR2,Lot Shape_IR3,Lot Shape_Reg,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Utilities_NoSeWa,Utilities_NoSewr,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,Land Slope_Mod,Land Slope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition 1_Feedr,Condition 1_Norm,Condition 1_PosA,Condition 1_PosN,Condition 1_RRAe,Condition 1_RRAn,Condition 1_RRNe,Condition 1_RRNn,Condition 2_Feedr,Condition 2_Norm,Condition 2_PosA,Condition 2_PosN,Condition 2_RRAe,Condition 2_RRAn,Condition 2_RRNn,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Roof Style_Gable,Roof Style_Gambrel,Roof Style_Hip,Roof Style_Mansard,Roof Style_Shed,Roof Matl_CompShg,Roof Matl_Membran,Roof Matl_Metal,Roof Matl_Roll,Roof Matl_Tar&Grv,Roof Matl_WdShake,Roof Matl_WdShngl,Exterior 1st_AsphShn,Exterior 1st_BrkComm,Exterior 1st_BrkFace,Exterior 1st_CBlock,Exterior 1st_CemntBd,Exterior 1st_HdBoard,Exterior 1st_ImStucc,Exterior 1st_MetalSd,Exterior 1st_Plywood,Exterior 1st_PreCast,Exterior 1st_Stone,Exterior 1st_Stucco,Exterior 1st_VinylSd,Exterior 1st_Wd Sdng,Exterior 1st_WdShing,Exterior 2nd_AsphShn,Exterior 2nd_Brk Cmn,Exterior 2nd_BrkFace,Exterior 2nd_CBlock,Exterior 2nd_CmentBd,Exterior 2nd_HdBoard,Exterior 2nd_ImStucc,Exterior 2nd_MetalSd,Exterior 2nd_Other,Exterior 2nd_Plywood,Exterior 2nd_PreCast,Exterior 2nd_Stone,Exterior 2nd_Stucco,Exterior 2nd_VinylSd,Exterior 2nd_Wd Sdng,Exterior 2nd_Wd Shng,Mas Vnr Type_BrkFace,Mas Vnr Type_CBlock,Mas Vnr Type_Stone,Exter Qual_Fa,Exter Qual_Gd,Exter Qual_TA,Exter Cond_Fa,Exter Cond_Gd,Exter Cond_Po,Exter Cond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Bsmt Qual_Fa,Bsmt Qual_Gd,Bsmt Qual_Po,Bsmt Qual_TA,Bsmt Cond_Fa,Bsmt Cond_Gd,Bsmt Cond_Po,Bsmt Cond_TA,Bsmt Exposure_Gd,Bsmt Exposure_Mn,Bsmt Exposure_No,BsmtFin Type 1_BLQ,BsmtFin Type 1_GLQ,BsmtFin Type 1_LwQ,BsmtFin Type 1_Rec,BsmtFin Type 1_Unf,BsmtFin Type 2_BLQ,BsmtFin Type 2_GLQ,BsmtFin Type 2_LwQ,BsmtFin Type 2_Rec,BsmtFin Type 2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Heating QC_Fa,Heating QC_Gd,Heating QC_Po,Heating QC_TA,Central Air_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Kitchen Qual_Fa,Kitchen Qual_Gd,Kitchen Qual_Po,Kitchen Qual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Sev,Functional_Typ,Fireplace Qu_Fa,Fireplace Qu_Gd,Fireplace Qu_Po,Fireplace Qu_TA,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Finish_RFn,Garage Finish_Unf,Garage Qual_Fa,Garage Qual_Gd,Garage Qual_Po,Garage Qual_TA,Garage Cond_Fa,Garage Cond_Gd,Garage Cond_Po,Garage Cond_TA,Paved Drive_P,Paved Drive_Y,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Typ

3.5.3 Creating additional features 

    - Create new features
    - Polynomial features 

In [50]:
df_auto_fe = df_onehot_encoded.copy()  

feature_summary = []  # To track new features

# Detect numeric & categorical variables
numeric_cols = df_auto_fe.select_dtypes(include=['int64', 'float64']).columns.drop('SalePrice', errors='ignore')

# Create age of building
date_cols = [col for col in df_auto_fe.columns if 'Year Built' in col ]
for col in date_cols:
    if df_auto_fe[col].dtype in [np.int64, np.float64]:  # Only numeric years
        df_auto_fe[col+'_Age'] = df_auto_fe['Yr Sold'] - df_auto_fe[col]
        feature_summary.append({
            'Feature': 'Age',
            'Formula': 'Yr Sold - '+col,
            'Reason': 'Captures age from year column'
        })

# Polynomial Features (degree=2 for numeric variables)
poly_features = PolynomialFeatures(degree=2, include_bias=False)
poly_array = poly_features.fit_transform(df_auto_fe[numeric_cols])

poly_df = pd.DataFrame(poly_array, 
                       columns=poly_features.get_feature_names_out(numeric_cols),
                       index=df_auto_fe.index)

# Avoid explosion: Keep only interaction terms (not squared terms) to control col count
interaction_cols = [col for col in poly_df.columns if ' ' in col]
df_auto_fe = pd.concat([df_auto_fe, poly_df[interaction_cols]], axis=1)

for col in interaction_cols:
    feature_summary.append({
        'Feature': col,
        'Formula': 'Interaction of numeric variables',
        'Reason': 'Captures non-linear relationships'
    })


feature_summary_df = pd.DataFrame(feature_summary)

# Show dataset with new features
df_auto_fe.head()


,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Street_Pave,Lot Shape_IR2,Lot Shape_IR3,Lot Shape_Reg,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Utilities_NoSeWa,Utilities_NoSewr,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,Land Slope_Mod,Land Slope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition 1_Feedr,Condition 1_Norm,Condition 1_PosA,Condition 1_PosN,Condition 1_RRAe,Condition 1_RRAn,Condition 1_RRNe,Condition 1_RRNn,Condition 2_Feedr,Condition 2_Norm,Condition 2_PosA,Condition 2_PosN,Condition 2_RRAe,Condition 2_RRAn,Condition 2_RRNn,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Roof Style_Gable,Roof Style_Gambrel,Roof Style_Hip,Roof Style_Mansard,Roof Style_Shed,Roof Matl_CompShg,Roof Matl_Membran,Roof Matl_Metal,Roof Matl_Roll,Roof Matl_Tar&Grv,Roof Matl_WdShake,Roof Matl_WdShngl,Exterior 1st_AsphShn,Exterior 1st_BrkComm,Exterior 1st_BrkFace,Exterior 1st_CBlock,Exterior 1st_CemntBd,Exterior 1st_HdBoard,Exterior 1st_ImStucc,Exterior 1st_MetalSd,Exterior 1st_Plywood,Exterior 1st_PreCast,Exterior 1st_Stone,Exterior 1st_Stucco,Exterior 1st_VinylSd,Exterior 1st_Wd Sdng,Exterior 1st_WdShing,Exterior 2nd_AsphShn,Exterior 2nd_Brk Cmn,Exterior 2nd_BrkFace,Exterior 2nd_CBlock,Exterior 2nd_CmentBd,Exterior 2nd_HdBoard,Exterior 2nd_ImStucc,Exterior 2nd_MetalSd,Exterior 2nd_Other,Exterior 2nd_Plywood,Exterior 2nd_PreCast,Exterior 2nd_Stone,Exterior 2nd_Stucco,Exterior 2nd_VinylSd,Exterior 2nd_Wd Sdng,Exterior 2nd_Wd Shng,Mas Vnr Type_BrkFace,Mas Vnr Type_CBlock,Mas Vnr Type_Stone,Exter Qual_Fa,Exter Qual_Gd,Exter Qual_TA,Exter Cond_Fa,Exter Cond_Gd,Exter Cond_Po,Exter Cond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Bsmt Qual_Fa,Bsmt Qual_Gd,Bsmt Qual_Po,Bsmt Qual_TA,Bsmt Cond_Fa,Bsmt Cond_Gd,Bsmt Cond_Po,Bsmt Cond_TA,Bsmt Exposure_Gd,Bsmt Exposure_Mn,Bsmt Exposure_No,BsmtFin Type 1_BLQ,BsmtFin Type 1_GLQ,BsmtFin Type 1_LwQ,BsmtFin Type 1_Rec,BsmtFin Type 1_Unf,BsmtFin Type 2_BLQ,BsmtFin Type 2_GLQ,BsmtFin Type 2_LwQ,BsmtFin Type 2_Rec,BsmtFin Type 2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Heating QC_Fa,Heating QC_Gd,Heating QC_Po,Heating QC_TA,Central Air_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Kitchen Qual_Fa,Kitchen Qual_Gd,Kitchen Qual_Po,Kitchen Qual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Sev,Functional_Typ,Fireplace Qu_Fa,Fireplace Qu_Gd,Fireplace Qu_Po,Fireplace Qu_TA,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Finish_RFn,Garage Finish_Unf,Garage Qual_Fa,Garage Qual_Gd,Garage Qual_Po,Garage Qual_TA,Garage Cond_Fa,Garage Cond_Gd,Garage Cond_Po,Garage Cond_TA,Paved Drive_P,Paved Drive_Y,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Typ

3.5.4 Scalling the numerical features using StandardScaler

    - Scaling helps with:
      * Preventing features with large ranges from dominating the model.
      * Improves convergence speed and model performance.
      * Essential for distance-based models (KNN, KMeans) and regularized models (Ridge, Lasso)


In [51]:
# Copy one-hot encoded dataframe
df_scaled = df_auto_fe.copy()

# Identify numeric columns (exclude target if present)

numeric_cols = df_scaled.select_dtypes(include=[np.number]).columns.drop('SalePrice', errors='ignore')

# Initialize scaler
scaler = StandardScaler()

# Fit and transform numeric columns
df_scaled[numeric_cols] = scaler.fit_transform(df_scaled[numeric_cols])

print("\n StandardScaler applied to numeric columns.")

# Keep track of scaling parameters

df_scaled.head()



 StandardScaler applied to numeric columns.


,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Street_Pave,Lot Shape_IR2,Lot Shape_IR3,Lot Shape_Reg,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Utilities_NoSeWa,Utilities_NoSewr,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,Land Slope_Mod,Land Slope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition 1_Feedr,Condition 1_Norm,Condition 1_PosA,Condition 1_PosN,Condition 1_RRAe,Condition 1_RRAn,Condition 1_RRNe,Condition 1_RRNn,Condition 2_Feedr,Condition 2_Norm,Condition 2_PosA,Condition 2_PosN,Condition 2_RRAe,Condition 2_RRAn,Condition 2_RRNn,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Roof Style_Gable,Roof Style_Gambrel,Roof Style_Hip,Roof Style_Mansard,Roof Style_Shed,Roof Matl_CompShg,Roof Matl_Membran,Roof Matl_Metal,Roof Matl_Roll,Roof Matl_Tar&Grv,Roof Matl_WdShake,Roof Matl_WdShngl,Exterior 1st_AsphShn,Exterior 1st_BrkComm,Exterior 1st_BrkFace,Exterior 1st_CBlock,Exterior 1st_CemntBd,Exterior 1st_HdBoard,Exterior 1st_ImStucc,Exterior 1st_MetalSd,Exterior 1st_Plywood,Exterior 1st_PreCast,Exterior 1st_Stone,Exterior 1st_Stucco,Exterior 1st_VinylSd,Exterior 1st_Wd Sdng,Exterior 1st_WdShing,Exterior 2nd_AsphShn,Exterior 2nd_Brk Cmn,Exterior 2nd_BrkFace,Exterior 2nd_CBlock,Exterior 2nd_CmentBd,Exterior 2nd_HdBoard,Exterior 2nd_ImStucc,Exterior 2nd_MetalSd,Exterior 2nd_Other,Exterior 2nd_Plywood,Exterior 2nd_PreCast,Exterior 2nd_Stone,Exterior 2nd_Stucco,Exterior 2nd_VinylSd,Exterior 2nd_Wd Sdng,Exterior 2nd_Wd Shng,Mas Vnr Type_BrkFace,Mas Vnr Type_CBlock,Mas Vnr Type_Stone,Exter Qual_Fa,Exter Qual_Gd,Exter Qual_TA,Exter Cond_Fa,Exter Cond_Gd,Exter Cond_Po,Exter Cond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Bsmt Qual_Fa,Bsmt Qual_Gd,Bsmt Qual_Po,Bsmt Qual_TA,Bsmt Cond_Fa,Bsmt Cond_Gd,Bsmt Cond_Po,Bsmt Cond_TA,Bsmt Exposure_Gd,Bsmt Exposure_Mn,Bsmt Exposure_No,BsmtFin Type 1_BLQ,BsmtFin Type 1_GLQ,BsmtFin Type 1_LwQ,BsmtFin Type 1_Rec,BsmtFin Type 1_Unf,BsmtFin Type 2_BLQ,BsmtFin Type 2_GLQ,BsmtFin Type 2_LwQ,BsmtFin Type 2_Rec,BsmtFin Type 2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Heating QC_Fa,Heating QC_Gd,Heating QC_Po,Heating QC_TA,Central Air_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Kitchen Qual_Fa,Kitchen Qual_Gd,Kitchen Qual_Po,Kitchen Qual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Sev,Functional_Typ,Fireplace Qu_Fa,Fireplace Qu_Gd,Fireplace Qu_Po,Fireplace Qu_TA,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Finish_RFn,Garage Finish_Unf,Garage Qual_Fa,Garage Qual_Gd,Garage Qual_Po,Garage Qual_TA,Garage Cond_Fa,Garage Cond_Gd,Garage Cond_Po,Garage Cond_TA,Paved Drive_P,Paved Drive_Y,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Typ

In [52]:
print(len(numeric_cols))
print(len(scaler.mean_))
print(len(scaler.scale_))

987
1057
1057
